In [1]:
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%precision 4

u'%.4f'

In [2]:
%load_ext rpy2.ipython
from rpy2.robjects.packages import importr
p1=importr('leaps')
p2=importr('stats')


In [64]:
# Wrote the source data
df = pd.read_csv('ncaa.data2.txt',delim_whitespace=True)
df.head()
x = df.ix[:,0:19]
y = df.ix[:,19]
# get rid of na's since regsubsets can't handle na's
x = x.dropna()
y = y.dropna()

m = x.shape[1]
n = x.shape[0]
if(m >= n): 
   m1=n-5  
else: 
   m1=m 
vm = range(m1)
# if only partially named columns corrects for no colnames
pvm = [0] * m1


In [65]:
%%R -i x,y,vm -o out_x
require(leaps)
regsubsets(x,y,method="forward")->out_x



In [66]:
rss = out_x[9]
nn = out_x[26][0]
q = [(rss[i]-rss[i+1])*(nn-i-1)/rss[i+1] for i in range(len(rss)-1)]
import scipy.stats as ss
rvf = [ ss.f(1,nn-i-1)  for i in range(len(rss)-1)]
orig =  [round(rvf[i].pdf(q[i]),5) for i in range(len(rss)-1)]
# sequential max of pvalues
for i in range(m1):
  pvm[i] = max(orig[0:i+1])
alpha = [0]+pvm
ng = len(alpha)
# will contain num. of true entering in orig
S = [0] * ng
# loop through alpha values for S=size                        
for ia in range(1,ng):                   
  S[ia] = sum([pvm[i] <= alpha[ia] for i in range(len(pvm))])        # size of models at alpha[ia], S[1]=0
ghat = [(m-S[i])*alpha[i]/(1+S[i]) for i in range(len(alpha))]              # gammahat_ER
# add additional points to make jumps
print ghat


[0, 0.0, 0.00017, 0.013533333333333333, 0.013533333333333333, 0.013533333333333333, 0.043847142857142855, 0.0438, 0.051398181818181815, 0.051398181818181815, 0.051398181818181815, 0.10648, 0.11042230769230768, 0.07784666666666666, 0.07784666666666666, 0.1134375, 0.12911823529411764, 0.10755111111111111, 0.11606315789473684, 0.0]


20


# Pseudocode/outline of algorithm


- Step1: Got rid of NA's in the data set 

- Step2: Wrote the forward selection method for the linear regression. The returned list includes the p-values in each iteration and the order of the varaibles entered
    - First, fit one predictor a time on the response y and choose the predictors which has the smallest p-value
    - Second, fit this predictor and one left predictor on the response y and choose the predictor which leads to the smalles pvalue. Up to now, we have two selected variables
    - Finally, after some steps, all the variables going to be entered bigger than 0.05, the forward selection method stops.
- Step3: Create the pvm vector which saves the max p_value according to the following formula and get the $ \hat{\gamma}$
    - First, caculate the pv.orig based on the following the formula 
    - Second, select the max_pvalue based on the formula 
      $\tilde{p_1} < \tilde{p_2} < \tilde{p_3}...< \tilde{p_k}$
    - Third, built the alpha vector as (0,pmv)
    - Fourth, create a S vector which stands for model size based on the formula
      $$ S = max \{i : \tilde{p_i} < \alpha \} $$
    - Calculate the
      $ \hat{\gamma} = \frac{(m-S)* \alpha}{1 + S}\\$  
      
- Step4: Determine the model size and returned the model
    
   $${\hat{\gamma} < \hat{\gamma_0}, \alpha <= \alpha_{max}}$$


# Unite Test:

  1. Whether there is missing data in the data set
  2. Test whether the result implmemented by this algorithm has the same answer with the paper

In [3]:
%load_ext rmagic


In [ ]:
%%R

install.packages('leaps')

Would you like to use a personal library instead?  (y/n) y
Would you like to create a personal library
~/R/x86_64-pc-linux-gnu-library/3.1
to install packages into?  (y/n) y
Selection: leaps
Selection: 
Selection: 
Selection: 
